In [1]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd
from rtree import Rtree

In [3]:
from keplergl import KeplerGl

In [2]:
df = gpd.read_file("NYPD_Complaint_2020.csv")
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,geometry
0,314773184,48,BRONX,12/31/2019,18:00:00,,,COMPLETED,,,...,,45-64,WHITE HISPANIC,F,1016990,244612,40.838026269000075,-73.88168118799997,"(40.838026269000075, -73.88168118799997)",None
1,289837961,25,MANHATTAN,12/30/2019,20:30:00,12/31/2019,10:00:00,COMPLETED,,,...,,25-44,WHITE HISPANIC,F,999296,230862,40.800334261000046,-73.94565697199994,"(40.800334261000046, -73.94565697199994)",None
2,535744284,77,BROOKLYN,12/24/2019,16:55:00,12/24/2019,17:00:00,COMPLETED,,,...,,18-24,WHITE,M,1001068,183317,40.66983179600004,-73.93937555099996,"(40.66983179600004, -73.93937555099996)",None
3,895678119,52,BRONX,12/30/2019,19:32:00,,,COMPLETED,,,...,,UNKNOWN,UNKNOWN,D,1009690,257590,40.87367103500002,-73.90801364899994,"(40.87367103500002, -73.90801364899994)",None
4,299841674,18,MANHATTAN,12/30/2019,15:30:00,12/30/2019,16:50:00,COMPLETED,,,...,,25-44,WHITE HISPANIC,M,985766,216503,40.76093528000007,-73.99452906599998,"(40.76093528000007, -73.99452906599998)",None


In [3]:
longitude = pd.to_numeric(df.Longitude)
latitude = pd.to_numeric(df.Latitude)

In [4]:
#Create a basemap 
map = KeplerGl(height=600, width=800)
# Create a gepdataframe

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(longitude, latitude))
# Add data to Kepler
map.add_data(data=gdf, name="crimes")
map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'crimes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [6]:
Neighborhood = gpd.read_file("Neighborhood Tabulation Areas (NTA).geojson")

Neighborhood.head()



,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005019.048,047,Borough Park,39247.2278309,Brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,QN51,52488277.846,081,Murray Hill,33266.9047967,Queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,QN27,19726845.6282,081,East Elmhurst,19816.7120736,Queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."
3,QN07,22887772.8652,081,Hollis,20976.3355739,Queens,4,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ..."
4,MN06,10647077.5264,061,Manhattanville,17040.6854129,Manhattan,1,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ..."


In [7]:
import osmnx as ox

In [8]:
def count_incidents_neighborhood(data, neighb):
 # spatial join and group by to get count of incidents in each poneighbourhood 
    joined = gpd.sjoin(gdf, neighb, op="within")
    grouped = joined.groupby('ntaname').size()
    df = grouped.to_frame().reset_index()
    df.columns = ['ntaname', 'count']
    merged = Neighborhood.merge(df, on='ntaname', how='outer')
    merged['count'].fillna(0,inplace=True)
    merged['count'] = merged['count'].astype(int)
    return merged

merged_gdf = count_incidents_neighborhood(gdf, Neighborhood)

c:\users\vibodha vimarshana\opencv\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [10]:
map2 = KeplerGl(height=600, width=800)
# Add data to Kepler
map2.add_data(data=merged_gdf, name="NeighborhoodCrimes")
map2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'NeighborhoodCrimes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

In [6]:
df_b = gpd.read_file("Building Footprints (2).geojson")

In [7]:
map3 = KeplerGl(height=600, width=800)
# Add data to Kepler
map3.add_data(data=df_b[:1000], name="Buildings")
map3

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Buildings': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…